# 自動化機器學習

使用scikit-learn的[數字數據集]（http://scikit-learn.org/stable/datasets/index.html#optical-recognition-of-handwriting-digits-dataset）展示如何使用 AutoML用於簡單的分類問題。

1.在現有的“工作區”中創建一個“實驗”。
2.使用“ AutoMLConfig”配置AutoML。
3.使用本地計算訓練模型。
4.探索結果。
5.測試最佳擬合模型。

## Import Class

作為安裝的一部分，已經創建了一個Azure ML Workspace對象。 
對於AutoML，需要創建一個Experiment對象，該對像是工作區中用於運行實驗的命名對象。

In [ ]:
import logging
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig

# 訪問Azure ML工作區需要使用Azure進行身份驗證。

In [ ]:
ws = Workspace.from_config()

# Choose a name for the experiment and specify the project folder.
experiment_name = 'automl-classification'
project_folder = './sample_projects/automl-classification'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

# 資料來源

使用scikit-learn的load_digits方法。

In [3]:
digits = datasets.load_digits()

# Exclude the first 100 rows from training so that they can be used for test.
X_train = digits.data[100:,:]
y_train = digits.target[100:]

## Train

實例化一個AutoMLConfig object以指定用於運行實驗的設置和數據。

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric主要的指標**|要優化的指標。 分類支持以下主要指標: <br><i>accuracy</i><br><i>AUC_weighted</i><br><i>average_precision_score_weighted</i><br><i>norm_macro_recall</i><br><i>precision_score_weighted</i>|
|**X**|(sparse) array-like, shape = [n_samples, n_features]|
|**y**|(sparse) array-like, shape = [n_samples, ], Multi-class targets.|
|**n_cross_validations**|Number of cross validation splits.|
|

自動化機器學習可訓練多個機器學習管道。 每次管道訓練都稱為iteration迭代(要執行訓練檔幾次)。

您可以使用迭代參數指定最大迭代次數。
您可以使用experiment_timeout_minutes參數指定運行的最長時間。
如果您既未指定迭代次數，也未指定experiment_timeout_minutes，則自動ML將繼續運行迭代，同時繼續查看分數的提高。
以下示例未指定迭代次數或experiment_timeout_minutes，因此一直運行直到分數不再提高。


# 不指定訓練幾次(最大迭代次數)也不指定運行的最長時間，一直運行直到分數不再提高

In [ ]:
automl_config = AutoMLConfig(task = 'classification',
                             primary_metric = 'AUC_weighted',
                             X = X_train, 
                             y = y_train,
                             n_cross_validations = 3)

在實驗 object 上調用Submit方法並傳遞運行的config。 本機運行的執行是同步的。 根據數據和迭代次數，這可能會運行一段時間。
在此示例中，我們指定show_output = True以將當前正在運行的迭代輸出到控制台。

In [ ]:
local_run = experiment.submit(automl_config, show_output = True)

In [ ]:
local_run

通過調用不帶迭代參數的continue_experiment繼續中斷的本地運行，或者通過指定迭代參數為完成的運行運行更多的迭代：

In [ ]:
local_run = local_run.continue_experiment(X = X_train, 
                                          y = y_train, 
                                          show_output = True,
                                          iterations = 5)

# 使用有指定最大迭代次數以及最大運行時間的config

In [6]:
automated_ml_config = AutoMLConfig(task = 'classification',
                             primary_metric = 'AUC_weighted',
                             X = X_train, 
                             y = y_train,
                             n_cross_validations = 3,
                             iterations=2,
                             iteration_timeout_minutes=5)


run = experiment.submit(automated_ml_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_d33cf7ad-fc0a-4122-92e4-06a5397b844d
Current status: DatasetCrossValidationSplit. Generating CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   StandardScalerWrapper SGD                      0:01:29       0.9941    0.9941
         1   StandardScalerWrapper SGD                      0:01:30       0.9970    0.9970


# 檢索所有子運行
還可以使用SDK方法獲取所有子運行，並查看我們記錄的各個指標。

In [10]:
children = list(run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}
    metricslist[int(properties['iteration'])] = metrics

rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

,0,1
AUC_macro,0.99,1.00
AUC_micro,0.99,1.00
AUC_weighted,0.99,1.00
accuracy,0.95,0.96
average_precision_score_macro,0.98,0.99
average_precision_score_micro,0.98,0.99
average_precision_score_weighted,0.98,0.99
balanced_accuracy,0.95,0.96
f1_score_macro,0.95,0.96
f1_score_micro,0.95,0.96
